In [ ]:
#setup - one-time tasks (run from terminal, not jupyter)
pip install cca-zoo

In [3]:
#import libraries
import pandas as pd
import numpy as np
import scipy
import os

userhome = os.path.expanduser('~')

In [4]:
# load residualized and scaled clinical features (from R script)
#replication
clin_file = userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/data/Data_prep/clin_features_scaled.csv'

clin_features = pd.read_csv(clin_file)
clin_features = clin_features.to_numpy()
print(type(clin_features))
print(clin_features.shape)

<class 'numpy.ndarray'>
(2926, 8)


In [5]:
# load residualized and scaled imaging features (from R script)
#replication
rsFC_file = userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/data/Data_prep/rsFC_features_scaled.csv'

rsFC_feat_chunks = pd.read_csv(rsFC_file, chunksize=1000)
rsFC_features = pd.concat(rsFC_feat_chunks)
#rsFC_features.sample(10)
rsFC_features = rsFC_features.to_numpy()
print(type(rsFC_features))
print(rsFC_features.shape)

<class 'numpy.ndarray'>
(2926, 61776)


In [13]:
# load selected rsFC features
#discovery
stable_features_file = userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/discovery/results/discovery_stable_feats_95_Pearson05.csv'
#replication
#stable_features_file = userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/replication_stable_feats_95_Pearson05.csv'

stable_features = pd.read_csv(stable_features_file, header=None)
stable_features = stable_features.to_numpy()
stable_features = np.transpose(stable_features)
print(type(stable_features))
print(stable_features.shape)

<class 'numpy.ndarray'>
(1, 7095)


In [14]:
# initialize arrays to store optimal hyperparams and scores from each CV loop
eCCA_eachfold_c1 = np.empty((10,27), dtype=object) 
eCCA_eachfold_c2 = np.empty((10,27), dtype=object) 
eCCA_eachfold_l1_ratio = np.empty((10,27), dtype=object)
eCCA_eachfold_score_TRAIN = np.empty((10,27), dtype=object) 
eCCA_eachfold_score_TEST = np.empty((10,27), dtype=object) 

print(eCCA_eachfold_c1)

[[None None None None None None None None None None None None None None
  None None None None None None None None None None None None None]
 [None None None None None None None None None None None None None None
  None None None None None None None None None None None None None]
 [None None None None None None None None None None None None None None
  None None None None None None None None None None None None None]
 [None None None None None None None None None None None None None None
  None None None None None None None None None None None None None]
 [None None None None None None None None None None None None None None
  None None None None None None None None None None None None None]
 [None None None None None None None None None None None None None None
  None None None None None None None None None None None None None]
 [None None None None None None None None None None None None None None
  None None None None None None None None None None None None None]
 [None None None Non

In [15]:
#hyperparam optimization
from cca_zoo.models import ElasticCCA

eCCA_model = ElasticCCA(max_iter=100000)
eCCA_param_grid = {"c" : [[1e-3,1e-2,1e-1], [1e-2,1e-1,3e-1]], "l1_ratio" : [0.25,0.5,0.75]}
# c is lasso alpha for each view
# l1_ratio is l1 ratio in lasso subproblems

for foldind in range(0,10):
    #foldind = 0 #remember python starts indexing at 0, while R and Matlab start at 1

    # load 10-fold cross validation (CV) splits
    #replication
    thisfold_file = userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/data/PartitionsEnCCA/partitions2_rep01fold' + str(foldind + 1) + r'.csv'
    print(thisfold_file)
    thisfold_file = pd.read_csv(thisfold_file)
    thisfold_file = thisfold_file.to_numpy()
    print(type(thisfold_file))
    print(thisfold_file.shape)

    # for each CV fold, extract only the selected participants and features
    # NB: put this in loop later once troubleshooting complete
    # NB: row and col indices were generated in R/Matlab, which index from 1. python indexes from 0
    thisfold_clin_features = clin_features[(thisfold_file - 1), :]
    #print(thisfold_clin_features.shape)
    thisfold_clin_features = np.squeeze(thisfold_clin_features)
    print(thisfold_clin_features.shape)
    #print(thisfold_clin_features)

    thisfold_rsFC_features = rsFC_features[(thisfold_file - 1), stable_features.astype(int)-1]
    print(thisfold_rsFC_features.shape)

    #thisfoldfeats = CVfold_features[foldind, :][~np.isnan(CVfold_features[foldind, :])]
    #thisfold_rsFC_features = rsFC_features[(thisfold_file - 1), thisfoldfeats.astype(int)-1]
    #print(thisfold_rsFC_features.shape)
    #print(thisfold_rsFC_features)


    #test set for this CV fold
    thisfold_clin_features_TEST = np.delete(clin_features,thisfold_file-1, axis=0)
    print(thisfold_clin_features_TEST.shape)
    thisfold_rsFC_features_TEST = np.delete(rsFC_features,thisfold_file-1, axis=0)
    thisfold_rsFC_features_TEST = thisfold_rsFC_features_TEST[:, stable_features.astype(int) - 1]
    thisfold_rsFC_features_TEST = np.squeeze(thisfold_rsFC_features_TEST)
    #thisfold_rsFC_features_TEST = thisfold_rsFC_features_TEST[:, thisfoldfeats.astype(int) - 1]
    print(thisfold_rsFC_features_TEST.shape)
    
    paramind = 0
    for thisc1 in eCCA_param_grid["c"][0]:
        for thisc2 in eCCA_param_grid["c"][1]:
            for thisl1_ratio in eCCA_param_grid["l1_ratio"]:
                eCCA_fit = ElasticCCA(c = (thisc1, thisc2), l1_ratio = thisl1_ratio, latent_dims=1).fit([thisfold_clin_features, thisfold_rsFC_features])
                eCCA_eachfold_c1[foldind,paramind] = thisc1
                eCCA_eachfold_c2[foldind,paramind] = thisc2
                eCCA_eachfold_l1_ratio[foldind,paramind] = thisl1_ratio
                print('params: ' + str(thisc1) + ', ' + str(thisc2) + ', ' + str(thisl1_ratio))
                eCCA_eachfold_score_TRAIN[foldind,paramind] = eCCA_fit.score((thisfold_clin_features, thisfold_rsFC_features))[0]
                print('CCA score (train): ' + str(eCCA_eachfold_score_TRAIN[foldind,paramind]))
                eCCA_eachfold_score_TEST[foldind,paramind] = eCCA_fit.score((thisfold_clin_features_TEST, thisfold_rsFC_features_TEST))[0]
                print('CCA score (test): ' + str(eCCA_eachfold_score_TEST[foldind,paramind]))

                paramind = paramind + 1

/Users/marshlab2/Desktop/OCS_bigdata_ML_local/FINAL/replication/data/PartitionsEnCCA/partitions2_rep01fold1.csv
<class 'numpy.ndarray'>
(2634, 1)
(2634, 8)
(2634, 7095)
(292, 8)
(292, 7095)


/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 0 not converged. Increase number of iterations.
  warnings.warn(


params: 0.001, 0.01, 0.25
CCA score (train): 0.9799550538846034
CCA score (test): 0.013357523997242104


/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 0 not converged. Increase number of iterations.
  warnings.warn(


params: 0.001, 0.01, 0.5
CCA score (train): 0.9509973758034038
CCA score (test): 0.024015047696634095


...

/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 0 not converged. Increase number of iterations.
  warnings.warn(


params: 0.1, 0.3, 0.5
CCA score (train): 0.12172856078029914
CCA score (test): 0.02632777698867983


/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:122: RuntimeWarning: invalid value encountered in true_divide
  - np.outer(score, score) @ residual / np.dot(score, score).item()


params: 0.1, 0.3, 0.75
CCA score (train): nan
CCA score (test): nan


/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [16]:
#scratch - check that output is working
eCCA_eachfold_score_TEST

array([[0.013357523997242104, 0.024015047696634095, 0.02387254337830269,
        0.06494228628703813, 0.08034762000959406, 0.10798699701271675,
        0.12444584244389345, 0.08597932757408677, nan,
        0.013559102295631398, 0.029059335097993078, 0.02491663518597864,
        0.07845811393921442, 0.10910731790413197, 0.13494631597653717,
        0.12501566422496047, 0.0870423873770807, nan,
        -0.006136543134060868, 0.023226432571726185,
        -0.025336165215354978, 0.05858600877003117, 0.09987000197796037,
        0.07943119655256825, 0.13248368284664602, 0.07239435000105865,
        nan],
       [0.021822279437953007, 0.018427258205131025, 0.03899354407002109,
        -0.00036620678165255605, 0.0718568661598038, 0.04000342232571574,
        0.04201501406959407, 0.007270660414080421, nan,
        0.02283531872582545, 0.014161055912492726, 0.029429752703832524,
        0.004415235917835103, 0.061215073398919184, 0.036540261802811225,
        0.03859629608133286, 0.00354934474

In [17]:
#average TRAIN and TEST correlations over CV folds

#print(eCCA_eachfold_score_TRAIN[0:4,:])
eCCA_average_score_TRAIN = np.nanmean(eCCA_eachfold_score_TRAIN[:,:], axis=0)
print(eCCA_average_score_TRAIN)

print('\n')

eCCA_average_score_TEST = np.nanmean(eCCA_eachfold_score_TEST[:,:], axis=0)
print(eCCA_average_score_TEST)
print(max(eCCA_average_score_TEST))

print('\n')
print(eCCA_eachfold_c1)
print('\n')
print(eCCA_eachfold_c2)
print('\n')
print(eCCA_eachfold_l1_ratio)

#generate visualization/heatmap of correlation by params

ZeroDivisionError: division by zero

In [19]:
# save files
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/discovery-selected-features/hyperparam_tuning/eCCA_eachfold_c1.csv', eCCA_eachfold_c1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/discovery-selected-features/hyperparam_tuning/eCCA_eachfold_c2.csv', eCCA_eachfold_c2, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/discovery-selected-features/hyperparam_tuning/eCCA_eachfold_l1_ratio.csv', eCCA_eachfold_l1_ratio, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/discovery-selected-features/hyperparam_tuning/eCCA_eachfold_score_TRAIN.csv', eCCA_eachfold_score_TRAIN, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/discovery-selected-features/hyperparam_tuning/eCCA_eachfold_score_TEST.csv', eCCA_eachfold_score_TEST, delimiter=',')

# END OF HYPERPARAMETER OPTIMIZATION CODE


In [20]:
#Rerun elastic net CCA using full discovery sample and optimized hyperparameters
from cca_zoo.models import ElasticCCA
eCCA_model = ElasticCCA(max_iter=1000000000)

#discovery-selected features
optim_c1 = 0.1
optim_c2 = 0.3
optim_l1_ratio = 0.25
#replication-selected features
#optim_c1 = 0.001
#optim_c2 = 0.01
#optim_l1_ratio = 0.75

#load selected feature indices file
allsubsample_selected_features = np.transpose(stable_features[:,:])
print(allsubsample_selected_features.shape)

allsubsample_rsFC_features = rsFC_features[:, allsubsample_selected_features[:,0] - 1]
#print(alldiscovery_rsFC_features)
print(allsubsample_rsFC_features.shape)

dims = min(np.linalg.matrix_rank(allsubsample_rsFC_features),np.linalg.matrix_rank(clin_features))
print('dims: ' + str(dims))

eCCA_fit = ElasticCCA(c = (optim_c1, optim_c2), l1_ratio = optim_l1_ratio, latent_dims=dims).fit([clin_features, allsubsample_rsFC_features])
print('params: ' + str(optim_c1) + ', ' + str(optim_c2) + ', ' + str(optim_l1_ratio))
allsubsample_scores = eCCA_fit.score((clin_features, allsubsample_rsFC_features))
print('All subsample CCA scores: ' + str(allsubsample_scores))

#calculate transformed canonical variate values
U,V = eCCA_fit.transform([clin_features, allsubsample_rsFC_features])
#U1,V1 = eCCA_fit.fit_transform([clin_features, alldiscovery_rsFC_features])



(7095, 1)
(2926, 7095)
dims: 8


/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 0 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 1 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 2 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 3 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 4 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 5 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaco

params: 0.1, 0.3, 0.25
All subsample CCA scores: [0.30785829 0.2588202  0.26790421 0.26115445 0.29509618 0.20255988
 0.25854365 0.26562124]


In [21]:
# View clinical loadings
#alldiscovery_loadings = eCCA_fit.get_loadings((clin_features, alldiscovery_rsFC_features))
allsubsample_loadings = eCCA_fit.get_loadings((clin_features, allsubsample_rsFC_features), normalize=True)
allsubsample_loadings_u = allsubsample_loadings[0]
allsubsample_loadings_v = allsubsample_loadings[1]

print(allsubsample_loadings_u.shape)
print(allsubsample_loadings_v.shape)

print('All subsample - clinical loadings CV1: \n' + str(allsubsample_loadings_u[:,0]))
print('All subsample - clinical loadings CV2: \n' + str(allsubsample_loadings_u[:,1]))
print('All subsample - clinical loadings CV3: \n' + str(allsubsample_loadings_u[:,2]))
print('All subsample - clinical loadings CV4: \n' + str(allsubsample_loadings_u[:,3]))
print('All subsample - clinical loadings CV5: \n' + str(allsubsample_loadings_u[:,4]))
print('All subsample - clinical loadings CV6: \n' + str(allsubsample_loadings_u[:,5]))
print('All subsample - clinical loadings CV7: \n' + str(allsubsample_loadings_u[:,6]))
print('All subsample - clinical loadings CV8: \n' + str(allsubsample_loadings_u[:,7]))

print(clin_features@allsubsample_loadings_u[:,0])


(8, 8)
(7095, 8)
All subsample - clinical loadings CV1: 
[-36.23461112 -23.26651035 -17.64034161 -18.27827868 -30.87635264
 -15.65411745 -15.93798782 -34.31117888]
All subsample - clinical loadings CV2: 
[  9.3414226   26.70461106  38.43223616  34.6054645    2.96560336
  -2.70985046 -14.55728883  17.91307486]
All subsample - clinical loadings CV3: 
[12.60788172  7.00976173  4.05623745  4.14789932 16.04268573 50.92832773
 19.75479543  7.94648911]
All subsample - clinical loadings CV4: 
[ -3.19814589  -5.82987734  18.01401483 -34.08248801  -3.37321765
  -1.03016694  -3.00194069  -2.88774666]
All subsample - clinical loadings CV5: 
[ -3.90879916   2.11663587   5.90217808   2.90245317 -40.08579763
  -9.53889899  -6.76321853  22.40986241]
All subsample - clinical loadings CV6: 
[19.14087767 54.74574608 20.0493424  22.71831906 13.43562577 14.94354819
 28.44842045 19.2110289 ]
All subsample - clinical loadings CV7: 
[18.12800489 10.34120401 25.31715538 20.45330288 15.92632801 20.59203329
 51.

In [22]:
# Save clin loadings
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_clinloadings_u_CV1norm.csv', allsubsample_loadings_u[:,0], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_clinloadings_u_CV2norm.csv', allsubsample_loadings_u[:,1], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_clinloadings_u_CV3norm.csv', allsubsample_loadings_u[:,2], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_clinloadings_u_CV4norm.csv', allsubsample_loadings_u[:,3], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_clinloadings_u_CV5norm.csv', allsubsample_loadings_u[:,4], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_clinloadings_u_CV6norm.csv', allsubsample_loadings_u[:,5], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_clinloadings_u_CV7norm.csv', allsubsample_loadings_u[:,6], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_clinloadings_u_CV8norm.csv', allsubsample_loadings_u[:,7], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_clinloadings_allnorm.csv', allsubsample_loadings_u, delimiter=',')



In [23]:
# View rs-FC loadings
allsubsample_loadings_v_CV1 = np.argsort(abs(allsubsample_loadings_v[:,0]))
#print(alldiscovery_loadings_v_CV1.shape)
print('CV1')
print(allsubsample_loadings_v_CV1[-20:] + 1)
print(allsubsample_loadings_v[allsubsample_loadings_v_CV1[-20:],0])

allsubsample_loadings_v_CV2 = np.argsort(abs(allsubsample_loadings_v[:,1]))
print('CV2')
print(allsubsample_loadings_v_CV2[-20:] + 1)
print(allsubsample_loadings_v[allsubsample_loadings_v_CV2[-20:],1])

allsubsample_loadings_v_CV3 = np.argsort(abs(allsubsample_loadings_v[:,2]))
print('CV3')
print(allsubsample_loadings_v_CV3[-20:] + 1)
print(allsubsample_loadings_v[allsubsample_loadings_v_CV3[-20:],2])

allsubsample_loadings_v_CV4 = np.argsort(abs(allsubsample_loadings_v[:,3]))
print('CV4')
print(allsubsample_loadings_v_CV4[-20:] + 1)
print(allsubsample_loadings_v[allsubsample_loadings_v_CV4[-20:],3])

allsubsample_loadings_v_CV5 = np.argsort(abs(allsubsample_loadings_v[:,4]))
allsubsample_loadings_v_CV6 = np.argsort(abs(allsubsample_loadings_v[:,5]))
allsubsample_loadings_v_CV7 = np.argsort(abs(allsubsample_loadings_v[:,6]))
allsubsample_loadings_v_CV8 = np.argsort(abs(allsubsample_loadings_v[:,7]))

CV1
[ 857  796  847 3465  347  350 2524  998  300 2051 3472 3249 3147 4210
  545  303   22 2135  537 1922]
[ -8.56942051  -8.63677465   8.67595258  -8.72420353  -8.76401239
  -9.29641937  -9.43080467  -9.44001602  -9.56158988  -9.60563173
  -9.68021515  -9.84486576 -10.04585609 -10.47377977 -10.60572908
 -10.82796689 -11.03556594 -11.04612459 -11.86851614 -13.27780614]
CV2
[5303 5269 1734 3456 4227 1870 5268 3467 4208 1717  415  396 6617 1869
  349 5371 1174  811 1876 5279]
[ 8.40205833  8.41959919  8.57862391  8.58194903  8.6342477   8.65066862
  8.72558863  8.73074393  8.87976333  8.89484759  8.90162788  8.90520553
  8.93514099  8.96268139  9.09284772  9.68094063 10.69536119 10.80273622
 11.03233733 11.30948912]
CV3
[5308 2149 2144 4931 2896 1842 1719 5396 4839 1723 4893 1066 4892 4934
 5487 1202 4917 1737 4886 4905]
[ -6.66086047   6.71116324   6.81531304   6.89992707   6.92103118
  -6.92511015  -7.01070296  -7.10965053   7.21819504   7.25606092
  -7.3051886    7.38681285  -7.660663

In [24]:
# Save rsFC loadings
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_rsFCinds_v_CV1.csv', allsubsample_loadings_v_CV1 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_rsFCloadings_v_CV1.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV1,0], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_rsFCinds_v_CV2.csv', allsubsample_loadings_v_CV2 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_rsFCloadings_v_CV2.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV2,1], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_rsFCinds_v_CV3.csv', allsubsample_loadings_v_CV3 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_rsFCloadings_v_CV3.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV3,2], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_rsFCinds_v_CV4.csv', allsubsample_loadings_v_CV4 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_rsFCloadings_v_CV4.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV4,3], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_rsFCinds_v_CV5.csv', allsubsample_loadings_v_CV5 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_rsFCloadings_v_CV5.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV5,4], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_rsFCinds_v_CV6.csv', allsubsample_loadings_v_CV6 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_rsFCloadings_v_CV6.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV6,5], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_rsFCinds_v_CV7.csv', allsubsample_loadings_v_CV7 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_rsFCloadings_v_CV7.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV7,6], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_rsFCinds_v_CV8.csv', allsubsample_loadings_v_CV8 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allreplication_rsFCloadings_v_CV8.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV8,7], delimiter=',')



In [25]:
# Save files
#discovery
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/transformed_clin_U.csv', U, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/transformed_rsFC_V.csv', V, delimiter=',')


In [32]:
alldiscovery_scores = eCCA_fit.score((clin_features, alldiscovery_rsFC_features))
print('All discovery CCA scores: ' + str(alldiscovery_scores))



NameError: name 'alldiscovery_rsFC_features' is not defined

In [41]:
np.corrcoef(U[:,1], V[:,1])

array([[1.      , 0.792935],
       [0.792935, 1.      ]])